In [1]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import os
import requests
import pandas as pd
from IPython.display import clear_output

In [2]:
furnish = load_model('models/furnish_classifier.keras')
bathroom = load_model('models/bathroom_classifier.keras')
exterior = load_model('models/exterior_classifier.keras')

In [7]:
def is_furnished(img): return furnish.predict(img) < 0.5
def is_bathroom(img): return bathroom.predict(img) < 0.5
def is_exterior(img): return exterior.predict(img) < 0.5

In [8]:
def download_image(url, save_path):
    try:
        response = requests.get(url, stream = True)
        if response.status_code == 200:
            with open(save_path, 'wb') as out_file:
                out_file.write(response.content)
            print(f"Successfully downloaded: {save_path}")
        else:
            print(f"Failed to download {url}")
    except Exception as e:
        print(f"Error occurred: {e}")

In [10]:
df = pd.read_excel('onMarketData.xlsx')
image_column = 'Image URLs'
save_directory = 'downloaded_images'

for i in range(0, len(df)): #loops thru all the image url cells
    print(i)
    if df[image_column][i] == '-': continue #skips if -, meaning blank
        
    links_arr = df[image_column][i].split() #splits them up into arr for easier processing

    for url in links_arr: #takes all the download links and downloads
        image_name = os.path.basename(url)
        save_path = os.path.join(save_directory, image_name)
        download_image(url, save_path)

    image_filenames = [file for file in os.listdir(save_directory) if file.endswith(('.png', '.jpg', '.jpeg'))] #all the file names

    interior_count = furnish_count = 0 #a count of how many rooms are interior
    
    
    for image_filename in image_filenames: #takes all the images and and runs it thru the model
        image_path = os.path.join(save_directory, image_filename) 
        
        # Load and preprocess the image for the cnn to be able to use
        img = load_img(image_path, target_size=(256, 256))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0

        if is_exterior(img_array): 
            continue #if its a exterior pic skip it

        if is_bathroom(img_array): 
            interior_count += 1
            continue #if bathroom pic increment interior and then skip
            
        if is_furnished(img_array): 
            furnish_count += 1 # if its furnished increm count

        interior_count += 1 #increments regardless of if its furnished or not
            
    percent_furnished = furnish_count / interior_count
    df.at[i, 'furnished'] = round(percent_furnished, 3) #writes the info
    clear_output(wait=True) #clears the consol for ease of use
    
df.to_excel('onMarketData.xlsx', index=False) #saves the info

print('all done')

1
Successfully downloaded: downloaded_images\item_2.jpg
Successfully downloaded: downloaded_images\item_7.jpg
Successfully downloaded: downloaded_images\thumb
Successfully downloaded: downloaded_images\item_9.jpg
Successfully downloaded: downloaded_images\item_19.jpg
Successfully downloaded: downloaded_images\item_8.jpg
Successfully downloaded: downloaded_images\item_10.jpg
Successfully downloaded: downloaded_images\item_11.jpg
Successfully downloaded: downloaded_images\item_54.jpg
Successfully downloaded: downloaded_images\item_14.jpg
Successfully downloaded: downloaded_images\item_32.jpg
Successfully downloaded: downloaded_images\item_12.jpg
Successfully downloaded: downloaded_images\item_33.jpg
Successfully downloaded: downloaded_images\item_21.jpg
Successfully downloaded: downloaded_images\item_22.jpg
Successfully downloaded: downloaded_images\item_25.jpg
Successfully downloaded: downloaded_images\item_23.jpg
Successfully downloaded: downloaded_images\item_24.jpg
Successfully downl

KeyboardInterrupt: 